<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_05_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# いつものDriveマウント

from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/[xxx]'

# スクレイピング

# 演習１

Webスクレイピング用ライブラリ

RequestsはWebサイトからHTMLデータの取得によく用いられる。
 
その後、取得したデータからBeautiful Soupなどのライブラリを用いて必要な情報のみを抽出する。

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://chonholee.github.io/tutorial/index.html'
response = requests.get(url)

In [ ]:
# https://developer.mozilla.org/ja/docs/Web/HTTP/Status

response.status_code

In [ ]:
response.encoding #文字コード

In [ ]:
response.content #取得した内容

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup

In [ ]:
soup.title

In [ ]:
soup.title.string

### HTML要素を取得する
### find_all() や select() などの使い方
https://gammasoft.jp/blog/difference-find-and-select-in-beautiful-soup-of-python/


In [ ]:
# ホームページのソースを見ながら、取得したい情報のタグを記入

# <body> 取得してみる

#--- here ---#
result = 
result

In [ ]:
# ページ内の全てのリンク(<a>)の名前とアドレスとを抽出する

#--- here ---#
for r in
    print()

In [ ]:
# ページ内の全てのリンクアドレス("href")をリストにする

#--- here ---#
links = 
links

### 例：株価の取得

In [ ]:
url = "https://www.nikkei.com/markets/kabu/"

#--- here ---#
res = 
soup = 

In [ ]:

#--- here ---#
price = 
print(price.text)

### 例：ヤフートップニュースの取得

In [ ]:
url = 'https://www.yahoo.co.jp/'

#--- here ---#
res = 
soup = 

# リンク取得して確認してみる
#--- here ---#
elems = 
elems

In [ ]:
import re

# 関連情報だけ抽出する
#--- here ---#
elems = 
elems

In [ ]:
elems[0].text

In [ ]:
elems[0].attrs['href']

In [ ]:
# 全トップニュースの見出しとリンクを出力
#--- here ---#
for elem in elems:
    print(elem.text, elem.attrs['href'])

# 演習２　テーブル

In [ ]:
url = 'https://chonholee.github.io/tutorial/bigdata/test-table.html'

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

soup

In [ ]:
# header 抽出
#--- here ---#
result = 
for a in result:
    print(a)

### 2-2　取得したデータをData Frameとして保存する

In [ ]:
import pandas as pd
import numpy as np

th_list = []
td_list = []
#--- here ---#
for a in soup.find_all(   ):
    th_list.append(a.string)

#--- here ---#
for a in soup.find_all(   ):
    td_list.append(a.string)

print(th_list)
print(td_list)

In [ ]:
# dataframeとして保存する

import pandas as pd
import numpy as np

df = pd.DataFrame(np.array([th_list, td_list]).T, columns=['項目', '値'])

df.head()

In [ ]:
df.to_csv('data.csv', index=False)

# 演習３　画像

In [ ]:
r = requests.get('https://chonholee.github.io/tutorial/bigdata/test-image.html')

soup = BeautifulSoup(r.content, 'html.parser')

soup

In [ ]:
# soup.find('img') # 一つ

soup.find_all('img') # 全部

In [ ]:
soup.find('img')['src']

In [ ]:
from PIL import Image
import io

page_dir = 'https://chonholee.github.io/tutorial/bigdata/'

img_url = soup.find('img')['src']

img_binary = io.BytesIO(requests.get(page_dir+img_url).content) #画像のURLから画像をバイナリーデータに変換

img = Image.open(img_binary) #バイナリーデータを画像化

In [ ]:
img

### 3-2 imgタグすべてを取得して画像を保存する

In [ ]:
img_tag_list = soup.find_all('img') #imgタグをすべて取得

img_tag_list

In [ ]:
for i, img_tag in enumerate(img_tag_list):

    img_url = img_tag['src']
    img_binary = io.BytesIO(requests.get(page_dir+img_url).content)
    img = Image.open(img_binary)
    
    img.save('{}.jpg'.format(i)) #保存

# 実践　プロ野球データ

In [ ]:
import requests
from bs4 import BeautifulSoup as BS

# HTMLを取得
url = 'https://baseball.yahoo.co.jp/npb/stats/batter?gameKindId=1'
res = requests.get(url)
content = res.text

In [ ]:
content

## htmlコードを見て、取得したいデータのタグを検証する

In [ ]:
# table 要素を取得
bs = BS(content, "lxml")  # parserの高速化 https://naruport.com/blog/2019/7/13/how-to-use-of-beautiful-soup-4/
tables = bs.find_all("table")
n_tables = len(tables)
print(n_tables)

In [ ]:
tables

In [ ]:
#抽出したテーブルが複数ある場合
if n_tables > 1:
    for i in range(n_tables):
        print(tables[i])            

In [ ]:
table = tables[0]

# table head 要素を取得 (存在する場合)
thead = table.find("thead")

# thead が存在する場合
if thead:
    tr = thead.find("tr")
    ths = tr.find_all("th")
    columns = [th.text for th in ths]    # pandas.DataFrame を意識
# thead が存在しない場合
else:
    columns = []

columns

In [ ]:
# table body 要素を取得
tbody = table.find("tbody")

# tr 要素を取得
trs = tbody.find_all("tr")

# 出力したい行データ
rows = [columns]

# td (th) 要素の値を読み込む
for tr in trs:
    row = [td.text for td in tr.find_all(["td"])]
    rows.append(row)

rows

In [ ]:
# データを書き込む
import csv

filepath = "baseball_data.csv"

with open(filepath, "w") as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerows(rows)

**簡単な方法**

pandas read_html で簡単にテーブル情報を取得できる

In [ ]:
import pandas as pd

url = 'https://baseball.yahoo.co.jp/npb/stats/batter?gameKindId=1'

dfs = pd.read_html(url, encoding='utf-8')[0]

In [ ]:
dfs

In [ ]:
dfs.to_csv('baseball_data_pandas.csv', encoding='utf-8', index=False, header=1)